In [1]:
import __future__
import codecs, os, re
from bs4 import BeautifulSoup
import requests
import ListRuns_cfg as L

# Testing area

## Testing original afs method

In [2]:
runs_afs=L.getruns_afs()

Fetching from: https://cmsweb.cern.ch/dqm/offline/data/browse/ROOT/OfflineData/
list of relative urls:  ['Run2018/ZeroBias/', 'Run2017/ZeroBias/', 'Run2016/ZeroBias/']
For Run2018/ZeroBias/
Out of a total of 106 run directories:
 44 have root files
 62 are empty
There are 401 runs with root files


For Run2017/ZeroBias/
Out of a total of 102 run directories:
 0 have root files
 102 are empty
There are 0 runs with root files


For Run2016/ZeroBias/
Out of a total of 104 run directories:
 61 have root files
 43 are empty
There are 410 runs with root files


For GLOBAL
Out of all run directories:
 105 have root files
 207 are empty
There are 811 runs with root files




### If you want to see parts of the list of runs just do 

In [3]:
# For runs WITH root files found on afs website
runs_afs[3:10]

[325169, 325168, 325159, 325114, 325111, 325100, 325099]

### Catching the repeated instances of the run numbers due to variations of the root file names

In [4]:
repeatruns=L.getrepeatedruns(runs_afs)

there are 67 repeated runs


# Lets use the JSON files to find the runs that we dont have

In [2]:
# Load the runs in a variable
list_runs , checkiter= L.getlist()

Found some runs
there are 0 repeated runs


In [26]:
def compare_with_Gjsn(runs):
    baseurl='https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/'
    json_runs_2018="Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt"
    json_runs_2017="Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt"
    json_runs_2016="Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt"


    json_runs=[json_runs_2018,json_runs_2017,json_runs_2016]#,url_runs_2015]
    print('\n',"Fetching from: "+baseurl)
    print('\n',"list of relative urls: ",json_runs)
    
    
    json_runslist=[]
    for url in json_runs:
        response=requests.get(baseurl+url) # open the document
        index = response.text                  # read the document
        a=str(index)                           # convert to string
        soup = BeautifulSoup(a, 'html.parser') # create the soup object

        a=re.findall(r"(\d{6})",str(soup))     #find exactly 6 occurrences of a number
                                               #in other words find 6 digit numbers
        print('---'*20)
        print('\n',len(a),"in",url)
        print('\n',"From",a[0],"to",a[-1])
        json_runslist.append(a)
    json_list = [int(item) for sublist in json_runslist for item in sublist]
    json_list.sort()
    good=[]
    missing=[]
    bad=[]
    for jrun in json_list:
        if jrun in list_runs:
            good.append(jrun)            
        else:
            missing.append(jrun)

    for run in list_runs:
        if run not in json_list:
            bad.append(grun)


                
    print('---'*20)
    print( '\n',len(good),"Good runs")
    print( '\n',len(missing),"Missing runs")
    print( '\n',len(bad),"bad runs")

    return json_list,good,missing,bad


In [28]:
json_list,good,missing,bad=compare_with_Gjsn(list_runs)



 Fetching from: https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/

 list of relative urls:  ['Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt', 'Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt', 'Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt']
------------------------------------------------------------

 478 in Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt

 From 315257 to 325172
------------------------------------------------------------

 474 in Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt

 From 297050 to 306460
------------------------------------------------------------

 393 in Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt

 From 273158 to 284044
----

In [32]:
for missrun in missing :
    if missrun in bad or missrun in good:
        print('flaw')
        
for badrun in bad:
    if badrun in missing or badrun in good:
        print('flaw')

for goodrun in good:
    if goodrun in missing or goodrun in bad:
        print('flaw')
                

# Lets test the RR API

In [60]:
def getRR(first=272000,last=326000,tkr_IN=True,tkr_strip_status='GOOD',tkr_pix_status='GOOD', name= 'UL',cl='Collisions'):
    runs = runregistry.get_datasets(filter={
#         'run_number': { 'and': [{'>': first}, {'<': last}]},
#         'tracker_included': tkr_IN,
#         'tracker-strip': tkr_strip_status,
        #'tracker-pixel': tkr_pix_status,
        'class': {  'like': '%{}%'.format(cl) },
#         'dataset_name':  {  'like': '%{}%'.format(name)  },
        })
        
    return runs

In [61]:
from ListRuns_cfg import runregistry

In [62]:
runs=getRR()

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verifica

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verifica

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is s

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verifica

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is s

...

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is s

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verifica

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'login.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advis

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate

/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.web.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/afs/cern.ch/user/g/gfidalgo/CMSSW_10_2_10/src/ML4TRKDQM/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsrunregistry.web.cern.ch'. Adding certificate verification is s

In [ ]:
len(runs)

previous runs

In [59]:
len(runs)

1554

In [44]:
runs[0].keys()

dict_keys(['class', 'state', 'short_run', 'significant', 'stop_reason', 'run_number', 'name', 'dataset_attributes', 'datasets_in_gui', 'deleted', 'version', 'lumisections'])

In [54]:
for RRrun in runs:
    print(RRrun['run_number'])

325175
325172
325170
325169
325168
325159
325117
325114
325113
325111
325110
325101
325100
325099
325098
325097
325057
325022
325001
325000
324999
324998
324997
324980
324970
324897
324878
324846
324841
324840
324835
324791
324785
324772
324769
324765
324764
324747
324729
324420
324318
324315
324293
324245
324237
324209
324207
324206
324205
324202
324201
324077
324022
324021
323997
323983
323980
323978
323976
323954
323940
323857
323841
323794
323790
323778
323775
323755
323727
323726
323725
323702
323696
323693
323526
323525
323524
323495
323493
323492
323488
323487
323475
323474
323473
323472
323471
323470
323423
323414
322633
322625
322617
322616
322605
322603
322602
322599
322510
322492
322480
322431
322430
322407
322381
322356
322355
322348
322332
322324
322322
322319
322317
322252
322222
322204
322201
322179
322118
322113
322106
322088
322079
322068
322057
322040
322022
322014
322013
321990
321988
321975
321973
321961
321960
321933
321919
321917
321909
321908
321887
321880
321879

# Golden Json Comparison with RR runs in txt file


In [33]:
list_runs , checkiter= L.getlist(file="Runs/AvailableRunsRR.txt",Zerobias=False)

Found some runs
there are 0 repeated runs


In [34]:
json_list,good,missing,bad=compare_with_Gjsn(list_runs)


 Fetching from: https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/

 list of relative urls:  ['Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt', 'Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt', 'Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt']
------------------------------------------------------------

 478 in Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt

 From 315257 to 325172
------------------------------------------------------------

 474 in Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt

 From 297050 to 306460
------------------------------------------------------------

 393 in Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt

 From 273158 to 284044
----

In [35]:
for missrun in missing :
    if missrun in bad or missrun in good:
        print('flaw')
        
for badrun in bad:
    if badrun in missing or badrun in good:
        print('flaw')

for goodrun in good:
    if goodrun in missing or goodrun in bad:
        print('flaw')
                